In [1]:
import sys

sys.path.append("..")

In [2]:
from transformers import (
    DataCollatorWithPadding,
    RobertaForSequenceClassification,
    RobertaTokenizerFast,
    TrainingArguments,
)

from adna.pylib import consts
from adna.pylib.bpe_dataset import BPEDataset
from adna.pylib.weighted_trainer import WeightedTrainer

## The model to fine tune

In [3]:
MODEL_PATH = consts.MT_DIR / "train" / "checkpoint-28443"

## Data augmentation parameters

In [4]:
REV_COMP_RATE = 0.5
TO_N_RATE = 0.02

## Training parameters

In [5]:
TRAIN_EPOCHS = 20
LEARNING_RATE = 1e-5
TRAIN_BATCH_SIZE = 192
EVAL_BATCH_SIZE = 192

MODEL_DIR = "finetune"  # Save check points to this sub-directory

## Get the tokenizer

In [6]:
tokenizer_path = str(consts.MT_DIR)
tokenizer = RobertaTokenizerFast.from_pretrained(tokenizer_path)

## Get the datasets

In [7]:
train_dataset = BPEDataset(
    "train",
    tokenizer,
    rev_comp_rate=REV_COMP_RATE,
    to_n_rate=TO_N_RATE,
)
eval_dataset = BPEDataset("val", tokenizer)

## Get the weights

In [8]:
WEIGHTS = train_dataset.get_weights()
WEIGHTS

array([0.60151492, 2.96269211])

## Get the model

In [9]:
model = RobertaForSequenceClassification.from_pretrained(
    MODEL_PATH, local_files_only=True
)

## Build the data collator

In [10]:
data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    padding="max_length",
    max_length=consts.MAX_LENGTH,
)

## Build the trainer

In [11]:
training_args = TrainingArguments(
    output_dir=consts.MT_DIR / MODEL_DIR,
    overwrite_output_dir=True,
    num_train_epochs=TRAIN_EPOCHS,
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=EVAL_BATCH_SIZE,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    seed=23,
)

In [12]:
trainer = WeightedTrainer(
    WEIGHTS,
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

## Train

In [ ]:
trainer.train()

/museum/rafe/other/adna/.venv/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 287233
  Num Epochs = 20
  Instantaneous batch size per device = 192
  Total train batch size (w. parallel, distributed & accumulation) = 192
  Gradient Accumulation steps = 1
  Total optimization steps = 29940


Epoch,Training Loss,Validation Loss


### It looks like epoch ?? is best

Path = ../data/UF46992/models/checkpoint-??
